# Fisheries Training Part 1 - Harvest Optimization and MOEA Diagnostics

This is the second post in a training series, studying decision making under deep uncertainty within the context of a complex harvested predator-prey fishery. 

The first post in this training series, [Fisheries Training 0: Exploring Predator-Prey Dynamics, can be found here](https://waterprogramming.wordpress.com/2022/07/11/__trashed-3/). 

The GitHub repository, containing all of the source code used throughout this series, is available [here](https://github.com/TrevorJA/harvested_predator_prey_system_tutorial_series). 

## Harvested predator-prey model

Re-vist the ODEs here.



## Harvest strategy objectives


#### Objective #1: 


#### Objective #2:


#### Objective #3:


#### Objective #4:

